In [1]:
%%capture
!git clone https://github.com/HackerBschor/SofteningQueryEvaluation
%cd SofteningQueryEvaluation

!pip3 install faiss-gpu-cu12
!pip3 install pgvector
!pip3 install kagglehub

In [2]:
from huggingface_hub import notebook_login
notebook_login()

# Evaluate Similarity Grouping


In [3]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel
from models.semantic_validation import LLaMAValidationModel

from db.operators import Dummy, Select
from db.criteria import SoftEqual
from db.structure import Column, Constant
import kagglehub

from evaluation.util import calculate_metrics, calc_bleu

import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
# Models
# meta-llama/Llama-3.2-3B-Instruct
# meta-llama/Meta-Llama-3-8B-Instruct

stem = SentenceTransformerEmbeddingModel(ModelMgr())
lsv = LLaMAValidationModel(ModelMgr(), model_path = "meta-llama/Meta-Llama-3-8B-Instruct")
# lsv = DeepSeekValidationModel(m)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [5]:
ZERO_SHOT_SYSTEM_PROMPT = "You are a validator. Respond with \"no\" and \"yes\" only!"
ZERO_SHOT_PROMPTING_TEMPLATE = 'Does "{}" describes "{}"'

random_state = 420
max_number_categories = 100
max_number_per_categories = 1

target = "category_1"

In [6]:
path = kagglehub.dataset_download("lakritidis/product-classification-and-categorization")

df = pd.read_csv(f"{path}/pricerunner_aggregate.csv", header=None, index_col=0)
df.drop(columns=[2, 3, 5], inplace=True)
df.columns = ["product", "category_1", "category_2"]
df.head()

100%|██████████| 22.8M/22.8M [00:02<00:00, 9.88MB/s]

Extracting files...


,product,category_1,category_2
0,,,
1,apple iphone 8 plus 64gb silver,Apple iPhone 8 Plus 64GB,Mobile Phones
2,apple iphone 8 plus 64 gb spacegrau,Apple iPhone 8 Plus 64GB,Mobile Phones
3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,Apple iPhone 8 Plus 64GB,Mobile Phones
4,apple iphone 8 plus 64gb space grey,Apple iPhone 8 Plus 64GB,Mobile Phones
5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,Apple iPhone 8 Plus 64GB,Mobile Phones


In [7]:
categories = pd.Series(df[target].unique())
categories = list(categories.sample(min(max_number_categories, len(categories)), random_state=random_state))

print(len(categories)), print(", ".join(categories)[:100], "...")

100
Fujifilm X-T2 + 18-135mm OIS, Russell Hobbs RHM2064R Red, Samsung WW70K5410UX, Intel Xeon Silver 410 ...


(None, None)

In [8]:
df_reduced = pd.concat([df[df[target] == category].sample(max_number_per_categories, random_state=random_state) for category in categories])

gt = {(x["product"], x[target]) for key, x in df_reduced.iterrows()}
print(str(gt)[0:100], "...", len(gt))

{('samsung addwash ww70k5410ux/eu washing machine graphite', 'Samsung WW70K5410UX'), ('intel xeon e5 ... 100


In [9]:
overall_result = {}

def evaluate(method, threshold, system_prompt=ZERO_SHOT_SYSTEM_PROMPT, prompt_template=ZERO_SHOT_PROMPTING_TEMPLATE):
    pred = []
    runtimes = []
    for category in tqdm(categories):
        d = Dummy("products", ["product"], [(x[0], ) for x in gt])
        s = Select(d, SoftEqual(Column("product"),Constant(category), method=method, em=stem, sv=lsv, threshold=threshold, zfs_system_prompt=system_prompt, zfs_prompt_template = prompt_template))

        tic = time.time()
        result = s.open().fetch_all()
        toc = time.time()

        pred.extend([(x["product"], category) for x in result])
        runtimes.append(toc - tic)

    scores = calculate_metrics(gt, set(pred), np.mean(runtimes))

    print(method, threshold, scores["F1 Score"])

    return scores, pred

In [10]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("threshold", thresh)
    overall_result[("threshold", thresh)] = res
    if res["Recall"] == 0.0:
        break

 59%|█████▉    | 59/100 [01:19<00:55,  1.35s/it]


KeyboardInterrupt: 

In [11]:
res, pred = evaluate("zero-few-shot", None)
overall_result[("zero-few-shot", None)] = res

100%|██████████| 100/100 [14:12<00:00,  8.53s/it]

zero-few-shot None 0.7206477732793523


In [12]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("both", thresh)
    overall_result[("both", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 100/100 [14:26<00:00,  8.66s/it]


both 0.1 0.7639484978540773


100%|██████████| 100/100 [10:55<00:00,  6.55s/it]


both 0.2 0.8148148148148148


100%|██████████| 100/100 [06:48<00:00,  4.08s/it]


...

both 0.8 0.11320754716981131


100%|██████████| 100/100 [02:09<00:00,  1.30s/it]


both 0.9 0.019801980198019802


100%|██████████| 100/100 [02:09<00:00,  1.30s/it]

both 1.0 0


In [13]:
for key in tqdm(overall_result):
    scores_bleu = calc_bleu(gt, overall_result[key]["pred"])
    for score_bleu in scores_bleu:
        overall_result[key][score_bleu] = scores_bleu[score_bleu]

100%|██████████| 11/11 [00:33<00:00,  3.05s/it]


In [14]:
df_results = pd.DataFrame([{"method": k[0], "threshold": k[1]} | v for k,v in overall_result.items()])
df_results

,method,threshold,Precision,Recall,F1 Score,tp,fn,fp,runtime,pred,bleu1,bleu2,bleu3,bleu4
0,zero-few-shot,NaN,0.605442,0.89,0.720648,89,11,58,8.528008,{(intel xeon e5 2630 2.2 ghz 10 core 20 thread...,0.930281,0.919245,0.909214,0.902544
1,both,0.1,0.669173,0.89,0.763948,89,11,44,8.658861,{(intel xeon e5 2630 2.2 ghz 10 core 20 thread...,0.929257,0.916011,0.904631,0.898332
2,both,0.2,0.758621,0.88,0.814815,88,12,28,6.553218,{(intel xeon e5 2630 2.2 ghz 10 core 20 thread...,0.920997,0.905815,0.893311,0.887122
3,both,0.3,0.871287,0.88,0.875622,88,12,13,4.080331,{(intel xeon e5 2630 2.2 ghz 10 core 20 thread...,0.918774,0.903292,0.891053,0.886190
4,both,0.4,0.905263,0.86,0.882051,86,14,9,2.516015,{(intel xeon e5 2630 2.2 ghz 10 core 20 thread...,0.906589,0.890170,0.876903,0.870213
5,both,0.5,0.924051,0.73,0.815642,73,27,6,1.833169,{(intel xeon e5 2630 2.2 ghz 10 core 20 thread...,0.819786,0.791529,0.766496,0.752481
6,both,0.6,0.946429,0.53,0.679487,53,47,3,1.522423,{(bosch geschirrsp ler exclusiv perfectdry smv...,0.676497,0.620095,0.578441,0.563414
7,both,0.7,1.000000,0.25,0.400000,25,75,0,1.348336,{(bosch geschirrsp ler exclusiv perfectdry smv...,0.446231,0.361033,0.302289,0.282627
8,both,0.8,1.000000,0.06,0.113208,6,94,0,1.310544,"{(samsung d500, Samsung SGH-D500), (intel xeon...",0.206630,0.128689,0.098889,0.085819
9,both,0.9,1.000000,0.01,0.019802,1,99,0,1.297444,"{(sony xperia c4 dual sim smartphone, Sony Xpe...",0.088904,0.040521,0.029910,0.026472


In [15]:
df_results.to_pickle("Products_fine_mpnetBaseV2_LLama8B.pkl")
df_results.drop(columns=["pred"]).to_csv("Products_fine_mpnetBaseV2_LLama8B.csv")